In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

In [2]:
# For local machine
df = pd.read_json('sparkify_data.json', lines=True)

# For Google colaboratory
# from google.colab import drive
# drive.mount('/content/gdrive/')
# df = pd.read_json('gdrive/My Drive/Deloitte Take Home/sparkify_data.json', lines=True)

## Preprocessing

In [3]:
df.dropna(inplace=True)

In [4]:
df['ts_converted'] = pd.to_datetime(df['ts'], unit="ms")

In [5]:
df['registration_converted'] = pd.to_datetime(df['registration'], unit="ms")

In [6]:
df['time_since_registered'] = (df['ts_converted'] - df['registration_converted']).dt.days

In [7]:
# 1 -> stayed in current level; 2-> converted from free to paid
subscribe_flow_df = df.groupby('userId')['level'].nunique().reset_index(drop=False)
subscribe_flow_df.columns = ['userId', 'subscribe_flow']
df = df.merge(subscribe_flow_df, left_on='userId', right_on='userId')

In [8]:
df['artist'] = df['artist'].str.lower().replace(" ", "_")

In [9]:
def summarise_user_agent(s):
    s = s.lower()
    if 'macintosh' in s:
        return 0
    elif 'windows' in s:
        return 1
    elif 'linux' in s:
        return 2
    else:
        return 3

In [10]:
df['userAgent_summarised'] = df.userAgent.apply(summarise_user_agent)

In [11]:
df['male'] = df.gender.apply(lambda x: 1 if x.lower() == 'm' else 0)

In [12]:
def get_state(s):
    s = s.lower()
    arr = s.split(',')
    return arr[-1].strip()

In [13]:
df['location_state']= df['location'].apply(get_state)

In [14]:
df.head()

,ts,userId,sessionId,page,auth,method,status,level,itemInSession,location,...,artist,song,length,ts_converted,registration_converted,time_since_registered,subscribe_flow,userAgent_summarised,male,location_state
0,1538352011000,293,292,NextSong,Logged In,PUT,200,free,20,"Corpus Christi, TX",...,martin orford,Grand Designs,597.55057,2018-10-01 00:00:11,2018-07-20 05:11:47,72,2,0,1,tx
1,1538352608000,293,292,NextSong,Logged In,PUT,200,free,21,"Corpus Christi, TX",...,the pussycat dolls,Santa Baby,180.50567,2018-10-01 00:10:08,2018-07-20 05:11:47,72,2,0,1,tx
2,1538352788000,293,292,NextSong,Logged In,PUT,200,free,22,"Corpus Christi, TX",...,eminem / hailie jade,My Dad's Gone Crazy,268.59057,2018-10-01 00:13:08,2018-07-20 05:11:47,72,2,0,1,tx
3,1538353056000,293,292,NextSong,Logged In,PUT,200,free,24,"Corpus Christi, TX",...,jedi mind tricks,Suicide,232.88118,2018-10-01 00:17:36,2018-07-20 05:11:47,72,2,0,1,tx
4,1538353288000,293,292,NextSong,Logged In,PUT,200,free,25,"Corpus Christi, TX",...,eagles of death metal,I Like To Move In The Night,239.75138,2018-10-01 00:21:28,2018-07-20 05:11:47,72,2,0,1,tx


## Data Exploration

In [15]:
df.describe()

,ts,sessionId,status,itemInSession,registration,length,time_since_registered,subscribe_flow,userAgent_summarised,male
count,4.328770e+05,432877.000000,432877.0,432877.000000,4.328770e+05,432877.000000,432877.000000,432877.000000,432877.000000,432877.000000
mean,1.540981e+12,2066.889853,200.0,109.646107,1.535525e+12,248.664593,62.661925,1.760181,0.779104,0.573341
std,1.477064e+09,1431.168033,0.0,117.858285,3.070613e+09,98.412670,39.216447,0.426973,0.781221,0.494592
min,1.538352e+12,1.000000,200.0,0.000000,1.509854e+12,0.783220,-1.000000,1.000000,0.000000,0.000000
25%,1.539745e+12,654.000000,200.0,27.000000,1.534368e+12,199.392200,36.000000,2.000000,0.000000,0.000000
50%,1.541020e+12,2016.000000,200.0,70.000000,1.536556e+12,234.004440,55.000000,2.000000,1.000000,1.000000
75%,1.542188e+12,3325.000000,200.0,151.000000,1.537612e+12,276.793020,79.000000,2.000000,1.000000,1.000000
max,1.543622e+12,4808.000000,200.0,1005.000000,1.543074e+12,3024.665670,390.000000,2.000000,3.000000,1.000000


In [16]:
df.columns

Index(['ts', 'userId', 'sessionId', 'page', 'auth', 'method', 'status',
       'level', 'itemInSession', 'location', 'userAgent', 'lastName',
       'firstName', 'registration', 'gender', 'artist', 'song', 'length',
       'ts_converted', 'registration_converted', 'time_since_registered',
       'subscribe_flow', 'userAgent_summarised', 'male', 'location_state'],
      dtype='object')

In [17]:
df.userAgent.nunique()

71

In [18]:
df.artist.nunique()

21245

In [19]:
df.song.nunique()

80292

In [20]:
df['userId'].nunique()

448

In [21]:
df['song'].nunique()

80292

In [22]:
# Average songs per userId
df.groupby('userId').song.count().median()

550.0

In [23]:
# Top songs per userId
df.groupby('userId').song.count().sort_values(ascending=False).head()

userId
92        8177
140       6233
300049    5879
101       5745
300035    5528
Name: song, dtype: int64

In [24]:
# Avg song length
df['length'].describe()

count    432877.000000
mean        248.664593
std          98.412670
min           0.783220
25%         199.392200
50%         234.004440
75%         276.793020
max        3024.665670
Name: length, dtype: float64

In [25]:
# Method column is not useful
df.method.value_counts()

PUT    432877
Name: method, dtype: int64

In [26]:
# Auth column is not useful
df['auth'].value_counts()

Logged In    432877
Name: auth, dtype: int64

In [27]:
# Page column is not useful
df['page'].value_counts()

NextSong    432877
Name: page, dtype: int64

In [28]:
# Status column is not useful
df['status'].value_counts()

200    432877
Name: status, dtype: int64

In [29]:
# level proportion for each unique userId
df.groupby('userId')['level'].agg(pd.Series.mode).value_counts()

paid    273
free    175
Name: level, dtype: int64

In [30]:
# Gender proportion for each unique userId
df.groupby('userId')['gender'].agg(pd.Series.mode).value_counts()

M    250
F    198
Name: gender, dtype: int64

In [31]:
# Most popular artist
df.groupby('artist').userId.count().sort_values(ascending=False).head()

artist
kings of leon             3497
coldplay                  3439
florence + the machine    2314
muse                      2194
dwight yoakam             2187
Name: userId, dtype: int64

In [32]:
# Most popular artist listened by free or paid users 
df.groupby(['artist','level']).userId.count().sort_values(ascending=False).head()

artist                  level
kings of leon           paid     2843
coldplay                paid     2793
florence + the machine  paid     1862
muse                    paid     1782
dwight yoakam           paid     1776
Name: userId, dtype: int64

In [33]:
# Most popular artist listened by free or paid users 
df.groupby(['song','artist']).userId.count().sort_values(ascending=False).head()

song                                                                 artist                                                                
You're The One                                                       dwight yoakam                                                             2166
Undo                                                                 bjãâ¶rk                                                                  1938
Revelry                                                              kings of leon                                                             1613
Sehr kosmisch                                                        harmonia                                                                  1341
Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile)  barry tuckwell/academy of st martin-in-the-fields/sir neville marriner    1236
Name: userId, dtype: int64

In [34]:
# How many songs do they listen in each session
df.groupby(['sessionId','userId']).song.count().mode()

0    5
dtype: int64

In [35]:
df.groupby('subscribe_flow').userId.nunique()

subscribe_flow
1    206
2    242
Name: userId, dtype: int64

## Model to predict propensity to subscribe

### Preprocessing & Train/test split

In [36]:
# processed_df = df.drop(columns=['auth','page','status','method','lastName','firstName','registration','ts'])
processed_df = df[['itemInSession', 'location_state', 'userAgent_summarised', 'male', 'length', 'time_since_registered', 'subscribe_flow']]

In [37]:
converted_df = processed_df[processed_df['subscribe_flow'] == 2].sample(10000, random_state=42)
unconverted_df = processed_df[processed_df['subscribe_flow'] == 1].sample(10000, random_state=42)
processed_df = pd.concat([converted_df, unconverted_df])

In [38]:
processed_df = pd.get_dummies(processed_df, prefix=['state_'], columns=['location_state'])

In [39]:
# Split train/test
from sklearn.model_selection import train_test_split

X = processed_df.drop(['subscribe_flow'], axis=1)
y = processed_df[['subscribe_flow']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=41)

### Training

In [40]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=1500, max_depth=10, random_state=0)
clf.fit(X_train, y_train)

<ipython-input-40-74092bac0dd7>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train, y_train)


RandomForestClassifier(max_depth=10, n_estimators=1500, random_state=0)

### Evaluation

In [41]:
y_pred=clf.predict(X_test)

In [44]:
from sklearn import metrics
print("F1 Score: ", metrics.f1_score(y_test, y_pred))

F1 Score:  0.8422971741112124
